In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from skimage import filters, morphology, measure
from scipy import ndimage
from tqdm import tqdm
import cv2

2025-05-02 12:56:31.179095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746190591.401151     936 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746190591.463849     936 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Use Mixed Precision (save VRAM)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
print("mixed precision enabled.")

mixed precision enabled.


In [3]:
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
# ----- MEMORY OPTIMIZATIONS -----
# Enable memory growth to avoid allocating all GPU memory at once
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("Memory growth enabled for GPUs")


Memory growth enabled for GPUs


In [5]:
# Set TensorFlow to only allocate necessary memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Memory growth needs to be set before GPUs have been initialized
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1746190600.782532     936 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [6]:
# Parameters
IMG_SIZE = 224  
BATCH_SIZE = 4
EPOCHS = 2
DATASET_ROOT = '/kaggle/input/preprocessed-mammo-splits'
OUTPUT_DIR = './cell_roi_extracted'
SEGMENTATION_MODEL_PATH = './cell_unet_model.keras'

In [7]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [8]:
def iou_metric(y_true, y_pred):
    # Ensure consistent data types by casting both to float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + tf.keras.backend.epsilon())

In [9]:
# Build U-Net model for cell segmentation
def build_unet_model(input_size=(IMG_SIZE, IMG_SIZE, 1)):
    inputs = Input(input_size)
    
    # Encoder (Downsampling)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    # Bridge
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder (Upsampling)
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)
    
    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)
    
    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)
    
    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
    
    # Output layer - sigmoid for binary segmentation
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', iou_metric])
    
    return model

In [10]:
def load_preprocessed_data():
    train_data = np.load(os.path.join(DATASET_ROOT, 'train_data.npz'))
    val_data = np.load(os.path.join(DATASET_ROOT, 'val_data.npz'))
    test_data = np.load(os.path.join(DATASET_ROOT, 'test_data.npz'))
    
    X_train, y_train = train_data['X'], train_data['y']
    X_val, y_val = val_data['X'], val_data['y']
    X_test, y_test = test_data['X'], test_data['y']
    
    # Ensure images are in grayscale for segmentation
    if X_train.shape[-1] == 3:
        X_train = np.mean(X_train, axis=-1, keepdims=True)
        X_val = np.mean(X_val, axis=-1, keepdims=True)
        X_test = np.mean(X_test, axis=-1, keepdims=True)
    
    print(f"Loaded data shapes: Train {X_train.shape}, Val {X_val.shape}, Test {X_test.shape}")
    return X_train, y_train, X_val, y_val, X_test, y_test

In [11]:
# Generator for batch-by-batch processing to save memory
def data_generator(images, masks=None, batch_size=BATCH_SIZE):
    num_samples = len(images)
    while True:
        # Shuffle at the beginning of each epoch
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]
            
            X_batch = images[batch_indices]
            
            if masks is not None:
                y_batch = masks[batch_indices]
                yield X_batch, y_batch
            else:
                yield X_batch

In [12]:
# Generate masks for cell images using adaptive thresholding
def generate_cell_masks(images, batch_size=32):
    masks = np.zeros_like(images)
    
    # Process in batches to avoid memory issues
    for i in tqdm(range(0, len(images), batch_size), desc="Generating cell masks"):
        batch_end = min(i + batch_size, len(images))
        batch_images = images[i:batch_end]
        
        for j, img in enumerate(batch_images):
            # Convert to single channel and scale to 0-255
            img_2d = img.squeeze()
            img_2d = (img_2d * 255).astype(np.uint8)
            
            # Apply adaptive thresholding
            binary = cv2.adaptiveThreshold(
                img_2d, 
                255, 
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                cv2.THRESH_BINARY_INV, 
                11, 
                2
            )
            
            # Clean up the mask with morphological operations
            kernel = np.ones((3, 3), np.uint8)
            opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
            
            # Remove small objects (noise)
            nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(opening, connectivity=8)
            sizes = stats[1:, -1]
            min_size = 30  # Minimum size of cell objects
            
            # Clean mask
            clean_mask = np.zeros_like(output)
            for k in range(1, nb_components):
                if sizes[k - 1] >= min_size:
                    clean_mask[output == k] = 1
            
            # Add channel dimension back
            masks[i+j] = clean_mask.reshape(img.shape)
    
    return masks

In [13]:
def extract_roi(images, masks, batch_size=32):
    roi_images = np.zeros_like(images)
    
    for i in tqdm(range(0, len(images), batch_size), desc="Extracting ROIs"):
        batch_end = min(i + batch_size, len(images))
        batch_images = images[i:batch_end]
        batch_masks = masks[i:batch_end]
        
        # Apply mask to original image
        roi_images[i:batch_end] = batch_images * batch_masks
    
    return roi_images

In [14]:
# Process individual cells for feature extraction
def extract_cell_features(image, mask):
    # Label individual cells
    labeled_mask = measure.label(mask.squeeze())
    props = measure.regionprops(labeled_mask)
    
    cell_features = []
    for prop in props:
        # Get bounding box
        min_row, min_col, max_row, max_col = prop.bbox
        
        # Extract cell
        cell = image.squeeze()[min_row:max_row, min_col:max_col]
        
        # Calculate features
        mean_intensity = prop.mean_intensity
        area = prop.area
        perimeter = prop.perimeter
        eccentricity = prop.eccentricity
        
        cell_features.append({
            'cell_image': cell,
            'mean_intensity': mean_intensity,
            'area': area,
            'perimeter': perimeter,
            'eccentricity': eccentricity
        })
    
    return cell_features

In [15]:
# Save some example segmentation results for visualization
def save_segmentation_examples(images, masks, rois, filename="cell_segmentation_examples.png"):
    fig, axes = plt.subplots(5, 3, figsize=(15, 25))
    
    for i in range(5):
        # Original image
        axes[i, 0].imshow(images[i].squeeze(), cmap='gray')
        axes[i, 0].set_title('Original Cell Image')
        axes[i, 0].axis('off')
        
        # Mask
        axes[i, 1].imshow(masks[i].squeeze(), cmap='gray')
        axes[i, 1].set_title('Cell Segmentation Mask')
        axes[i, 1].axis('off')
        
        # ROI
        axes[i, 2].imshow(rois[i].squeeze(), cmap='gray')
        axes[i, 2].set_title('Extracted Cell ROI')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, filename))
    plt.close()

In [16]:
# Create a visualization showing cell segmentation outlines
def save_cell_contour_visualization(images, masks, filename="cell_contours.png"):
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    for i in range(min(9, len(images))):
        row, col = i // 3, i % 3
        
        # Get original image
        img = images[i].squeeze()
        mask = masks[i].squeeze()
        
        # Create RGB visualization
        img_rgb = np.stack([img, img, img], axis=-1)
        
        # Find contours
        contours, _ = cv2.findContours(
            (mask * 255).astype(np.uint8),
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        # Draw contours on RGB image
        cv2.drawContours(img_rgb, contours, -1, (0, 1, 0), 1)  # Green contours
        
        # Display
        axes[row, col].imshow(img_rgb)
        axes[row, col].set_title(f'Cell Sample {i+1}')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, filename))
    plt.close()

In [17]:
def predict_in_batches(model, images, batch_size=BATCH_SIZE):
    num_samples = len(images)
    predictions = np.zeros_like(images)
    
    for i in tqdm(range(0, num_samples, batch_size), desc="Predicting"):
        batch_end = min(i + batch_size, num_samples)
        batch_images = images[i:batch_end]
        
        batch_predictions = model.predict(batch_images, verbose=0)
        predictions[i:batch_end] = batch_predictions
        
        # Clear GPU memory after each batch
        tf.keras.backend.clear_session()
    
    return predictions

In [18]:
def main():
    # Load preprocessed data
    print("Loading preprocessed data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_preprocessed_data()
    
    # Create or load U-Net model
    if os.path.exists(SEGMENTATION_MODEL_PATH):
        print(f"Loading existing U-Net model from {SEGMENTATION_MODEL_PATH}")
        custom_objects = {'iou_metric': iou_metric}
        model = tf.keras.models.load_model(SEGMENTATION_MODEL_PATH, custom_objects=custom_objects)
    else:
        print("Building and training lightweight U-Net segmentation model...")
        model = build_unet_model(input_size=(IMG_SIZE, IMG_SIZE, 1))
        
        
        # Generate training masks for a subset to save memory
        print("Generating cell masks...")
        # Use 50% of training data to reduce memory usage
        train_subset_size = len(X_train) // 2
        train_subset = X_train[:train_subset_size]
        train_masks = generate_cell_masks(train_subset)
        val_masks = generate_cell_masks(X_val)
        
        # Set up callbacks
        callbacks = [
            ModelCheckpoint(SEGMENTATION_MODEL_PATH, save_best_only=True, monitor='val_loss'),
            EarlyStopping(patience=5, monitor='val_loss'),  # Reduced patience
            ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6, monitor='val_loss')  # Modified
        ]
        
        # Create data generators
        train_gen = data_generator(train_subset, train_masks, batch_size=BATCH_SIZE)
        val_gen = data_generator(X_val, val_masks, batch_size=BATCH_SIZE)
        
        # Train the model with generators
        history = model.fit(
            train_gen,
            steps_per_epoch=len(train_subset) // BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=val_gen,
            validation_steps=len(X_val) // BATCH_SIZE,
            callbacks=callbacks
        )
        
        # Plot training history
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Loss')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'cell_training_history.png'))
        plt.close()
    
    # Predict segmentation masks in batches
    print("Predicting cell segmentation masks in batches...")
    
    # Process smaller subsets of the data to avoid OOM errors
    # For example, use 25% of training data for visualization
    train_viz_size = min(100, len(X_train))
    val_viz_size = min(100, len(X_val))
    test_viz_size = min(100, len(X_test))
    
    train_pred_masks = predict_in_batches(model, X_train[:train_viz_size])
    val_pred_masks = predict_in_batches(model, X_val[:val_viz_size])
    test_pred_masks = predict_in_batches(model, X_test[:test_viz_size])
    
    # Clear memory
    tf.keras.backend.clear_session()
    
    # Convert predictions to binary masks
    threshold = 0.5
    print("Converting predictions to binary masks...")
    train_binary_masks = (train_pred_masks > threshold).astype(np.float32)
    val_binary_masks = (val_pred_masks > threshold).astype(np.float32)
    test_binary_masks = (test_pred_masks > threshold).astype(np.float32)
    
    # Post-process the binary masks to clean them up
    print("Post-processing masks...")
    for i in range(len(train_binary_masks)):
        # Remove small objects
        binary = morphology.remove_small_objects(train_binary_masks[i].squeeze().astype(bool), min_size=30)
        # Fill holes
        binary = morphology.binary_closing(binary, morphology.disk(3))
        binary = ndimage.binary_fill_holes(binary)
        train_binary_masks[i] = binary.astype(np.float32).reshape(train_binary_masks[i].shape)
    
    for i in range(len(val_binary_masks)):
        binary = morphology.remove_small_objects(val_binary_masks[i].squeeze().astype(bool), min_size=30)
        binary = morphology.binary_closing(binary, morphology.disk(3))
        binary = ndimage.binary_fill_holes(binary)
        val_binary_masks[i] = binary.astype(np.float32).reshape(val_binary_masks[i].shape)
    
    for i in range(len(test_binary_masks)):
        binary = morphology.remove_small_objects(test_binary_masks[i].squeeze().astype(bool), min_size=30)
        binary = morphology.binary_closing(binary, morphology.disk(3))
        binary = ndimage.binary_fill_holes(binary)
        test_binary_masks[i] = binary.astype(np.float32).reshape(test_binary_masks[i].shape)
    
    # Extract ROIs in batches
    print("Extracting cell ROIs...")
    train_roi = extract_roi(X_train[:train_viz_size], train_binary_masks)
    val_roi = extract_roi(X_val[:val_viz_size], val_binary_masks)
    test_roi = extract_roi(X_test[:test_viz_size], test_binary_masks)
    
    # Save visualizations before full dataset processing
    print("Saving cell segmentation examples...")
    save_segmentation_examples(X_train[:5], train_binary_masks[:5], train_roi[:5], "train_cell_segmentation.png")
    save_segmentation_examples(X_val[:5], val_binary_masks[:5], val_roi[:5], "val_cell_segmentation.png")
    save_segmentation_examples(X_test[:5], test_binary_masks[:5], test_roi[:5], "test_cell_segmentation.png")
    
    # Save contour visualization
    save_cell_contour_visualization(X_train[:9], train_binary_masks[:9], "cell_contours_train.png")
    
    # Calculate segmentation metrics on a small sample
    print("Calculating segmentation metrics...")
    sample_size = min(50, len(val_binary_masks))
    
    # Generate reference masks for a subset
    reference_masks = generate_cell_masks(X_val[:sample_size])
    
    dice_scores = []
    iou_scores = []
    
    for i in range(sample_size):
        pred = val_binary_masks[i].squeeze().astype(np.bool_)
        true = reference_masks[i].squeeze().astype(np.bool_)
        
        intersection = np.logical_and(pred, true).sum()
        union = np.logical_or(pred, true).sum()
        
        # IoU
        iou = intersection / (union + 1e-7)
        iou_scores.append(iou)
        
        # Dice coefficient
        dice = (2 * intersection) / (pred.sum() + true.sum() + 1e-7)
        dice_scores.append(dice)
    
    # Save metrics
    with open(os.path.join(OUTPUT_DIR, 'segmentation_metrics.txt'), 'w') as f:
        f.write(f"Mean IoU: {np.mean(iou_scores):.4f}\n")
        f.write(f"Mean Dice: {np.mean(dice_scores):.4f}\n")
    
    # If memory allows, process the full datasets
    print("Processing full datasets in batches...")
    
    # Create an optimized function that processes full datasets in chunks and saves immediately
    def process_and_save_full_dataset(X, y, dataset_name, batch_size=16):
        """Process a full dataset in small batches and save immediately to avoid memory issues"""
        total_batches = len(X) // batch_size + (1 if len(X) % batch_size > 0 else 0)
        
        # Create NPZ files for appending
        output_file = os.path.join(OUTPUT_DIR, f'cell_roi_{dataset_name}_data.npz')
        
        # Process in batches
        for batch_idx in tqdm(range(total_batches), desc=f"Processing {dataset_name} dataset"):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, len(X))
            
            # Get batch
            X_batch = X[start_idx:end_idx]
            y_batch = y[start_idx:end_idx]
            
            # Predict masks
            pred_masks = model.predict(X_batch, verbose=0)
            binary_masks = (pred_masks > threshold).astype(np.float32)
            
            # Post-process masks
            for i in range(len(binary_masks)):
                binary = morphology.remove_small_objects(binary_masks[i].squeeze().astype(bool), min_size=30)
                binary = morphology.binary_closing(binary, morphology.disk(3))
                binary = morphology.binary_fill_holes(binary)
                binary_masks[i] = binary.astype(np.float32).reshape(binary_masks[i].shape)
            
            # Extract ROIs
            roi_batch = X_batch * binary_masks
            
            # Save this batch to a temporary file
            batch_file = os.path.join(OUTPUT_DIR, f'batch_{dataset_name}_{batch_idx}.npz')
            np.savez_compressed(batch_file, X=roi_batch, y=y_batch)
            
            # Clear memory
            del pred_masks, binary_masks, roi_batch
            tf.keras.backend.clear_session()
        
        # Combine all batch files into one dataset file
        combined_X = []
        combined_y = []
        
        for batch_idx in range(total_batches):
            batch_file = os.path.join(OUTPUT_DIR, f'batch_{dataset_name}_{batch_idx}.npz')
            if os.path.exists(batch_file):
                data = np.load(batch_file)
                combined_X.append(data['X'])
                combined_y.append(data['y'])
                # Delete temporary file
                os.remove(batch_file)
        
        # Save combined dataset
        combined_X = np.concatenate(combined_X, axis=0)
        combined_y = np.concatenate(combined_y, axis=0)
        np.savez_compressed(output_file, X=combined_X, y=combined_y)
        
        print(f"Saved {dataset_name} dataset with {len(combined_X)} samples")
    
    # Try to process the full datasets
    try:
        # Use very small batch size for full dataset processing
        full_batch_size = 8
        
        # Process with error handling
        try:
            process_and_save_full_dataset(X_train, y_train, 'train', batch_size=full_batch_size)
        except Exception as e:
            print(f"Error processing train dataset: {e}")
            print("Saving only visualized subset instead")
            np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_train_data.npz'), 
                                X=train_roi, y=y_train[:train_viz_size])
        
        try:
            process_and_save_full_dataset(X_val, y_val, 'val', batch_size=full_batch_size)
        except Exception as e:
            print(f"Error processing validation dataset: {e}")
            print("Saving only visualized subset instead")
            np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_val_data.npz'), 
                                X=val_roi, y=y_val[:val_viz_size])
        
        try:
            process_and_save_full_dataset(X_test, y_test, 'test', batch_size=full_batch_size)
        except Exception as e:
            print(f"Error processing test dataset: {e}")
            print("Saving only visualized subset instead")
            np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_test_data.npz'), 
                                X=test_roi, y=y_test[:test_viz_size])
    
    except Exception as e:
        print(f"Error in full dataset processing: {e}")
        print("Saving visualization subsets only")
        np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_train_data.npz'), X=train_roi, y=y_train[:train_viz_size])
        np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_val_data.npz'), X=val_roi, y=y_val[:val_viz_size])
        np.savez_compressed(os.path.join(OUTPUT_DIR, 'cell_roi_test_data.npz'), X=test_roi, y=y_test[:test_viz_size])
    
    print("Cell ROI extraction complete! Results saved to:", OUTPUT_DIR)
    print("You can now use the extracted ROIs for your transfer learning hybrid approach.")


In [19]:
if __name__ == "__main__":
    main()

Loading preprocessed data...
Loaded data shapes: Train (17755, 224, 224, 1), Val (3134, 224, 224, 1), Test (3687, 224, 224, 1)
Building and training lightweight U-Net segmentation model...
Generating cell masks...


Generating cell masks: 100%|██████████| 98/98 [00:04<00:00, 20.10it/s]

Epoch 1/2



I0000 00:00:1746190676.844151     969 service.cc:148] XLA service 0x7f9394028a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746190676.844991     969 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1746190678.303536     969 cuda_dnn.cc:529] Loaded cuDNN version 90300


   3/2219 ━━━━━━━━━━━━━━━━━━━━ 2:00 55ms/step - accuracy: 0.9852 - iou_metric: 0.0082 - loss: 0.6796 

...

Error processing train dataset: module 'skimage.morphology' has no attribute 'binary_fill_holes'
Saving only visualized subset instead


Processing val dataset:   0%|          | 0/392 [00:00<?, ?it/s]


Error processing validation dataset: module 'skimage.morphology' has no attribute 'binary_fill_holes'
Saving only visualized subset instead


Processing test dataset:   0%|          | 0/461 [00:00<?, ?it/s]


Error processing test dataset: module 'skimage.morphology' has no attribute 'binary_fill_holes'
Saving only visualized subset instead
Cell ROI extraction complete! Results saved to: ./cell_roi_extracted
You can now use the extracted ROIs for your transfer learning hybrid approach.
